### Example 2 - AI Attachment Reviewer

This example will walk you through the following steps:

1. Downloading attachments from a Feature Service
2. Use a LLM to analyze attachments (images & PDFs) and provide a text summary for each
3. Write that summary back to the feature for review in an ArcGIS Manager Instant Application


#### Download Feature Service Attachments

This helpful section of code was pulled from Rami's excellent sample here: https://github.com/ralouta/ArcGIS_Code_Repo/blob/main/src/scripts/Feature%20Service%20Management/download_attachments_fs.ipynb.


In [ ]:
import toml
import sys
import os
import shutil
from arcgis.gis import GIS
from langchain_openai import AzureChatOpenAI

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from chains.image_extractor import ImageExtractChain
from chains.pdf_extractor import PDFExtractChain
from models.image_information import ImageInformation
from models.pdf_information import PDFInformation

In [ ]:
# Connect to Esri Federal AGOL
gis = GIS("https://esrifederal.maps.arcgis.com", client_id="VEFjlNUX3GINnELq")

In [ ]:
# clone over a feature service and a webmap to your own content in a new folder
source_group = gis.groups.get("78a63c27e59941bbbff470a3f62e2d92")
source_group_items = source_group.content()

folder_name = "SE Led Training GeoGenAI 2024"
gis.content.create_folder(folder_name)
gis.content.clone_items(items=source_group_items, copy_data=True, folder=folder_name)

print(
    "done cloning. please check your Content for a new folder and then make sure you locate the Item Id of the new feature service that was created. You will use that for the next step."
)

In [ ]:
# Get the feature layer item id from the previous cloning step.
item_id = "ac062cbb932b478f96075b4ae326447c"

# Get the item
item = gis.content.get(item_id)

feature_layer = item.layers[0]

# Create a directory to save attachments
output_dir = "attachments"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Function to download attachments
def download_attachments(feature_layer, output_dir):
    # Query all features
    features = feature_layer.query(where="1=1", out_fields="*").features

    for feature in features:
        object_id = feature.attributes[feature_layer.properties.objectIdField]
        globalid = feature.attributes[feature_layer.properties.globalIdField]
        attachments = feature_layer.attachments.get_list(object_id)

        for attachment in attachments:
            attachment_id = attachment["id"]
            attachment_name = attachment["name"]
            final_attachment_path = os.path.join(
                output_dir, f"{globalid}__{attachment_name}"
            )

            # Check if the attachment already exists
            if not os.path.exists(final_attachment_path):
                temp_dir = os.path.join(output_dir, f"temp_{object_id}")

                # Create a temporary directory to download the attachment
                if not os.path.exists(temp_dir):
                    os.makedirs(temp_dir)

                # Download the attachment to the temporary directory
                feature_layer.attachments.download(
                    oid=object_id, attachment_id=attachment_id, save_path=temp_dir
                )

                # Move the attachment from the temporary directory to the output directory with the new name
                temp_attachment_path = os.path.join(temp_dir, attachment_name)
                shutil.move(temp_attachment_path, final_attachment_path)

                # Remove the temporary directory
                shutil.rmtree(temp_dir)

                print(
                    f"Downloaded {attachment_name} for feature {object_id} as {final_attachment_path}"
                )
            else:
                print(
                    f"Attachment {attachment_name} for feature {object_id} already exists as {final_attachment_path}"
                )


# Download attachments
download_attachments(feature_layer, output_dir)

### Use a LLM to analyze the attachments


In [ ]:
azure_config = toml.load("config.toml")["configs"][0]
llm = AzureChatOpenAI(
    openai_api_version=azure_config["api_version"],
    azure_deployment=azure_config["deployment_name"],
    api_key=azure_config["api_key"],
    azure_endpoint=azure_config["api_endpoint"],
    model=azure_config["model_name"],
    model_name=azure_config["model_name"],
    temperature=0,
)
llm.invoke("hi")

In [ ]:
# loop through attachments folder
for root, dirs, files in os.walk(output_dir):
    for file in files:
        global_id = file.split("__")[0]
        print("=====================================")
        print(f"Processing attachment for global id: {global_id}")

        file_path = os.path.join(root, file)

        attachment_description = None
        if file.upper().endswith(".PNG") or file.upper().endswith(".JPG"):
            print("analyzing image attachment ...")
            iec_chain = ImageExtractChain(model=llm, image_path=file_path)
            result = iec_chain.extract_from_image()
            attachment_description = iec_chain.format_for_attachment(result)
        else:
            print("analyzing pdf attachment ...")
            pdf_chain = PDFExtractChain(model=llm, pdf_path=file_path)
            result = pdf_chain.extract_from_pdf()
            attachment_description = pdf_chain.format_for_attachment(result)

        if attachment_description is None:
            continue

        feature = [
            {
                "attributes": {
                    feature_layer.properties.globalIdField: global_id,
                    "AIReview": attachment_description,
                }
            }
        ]

        print("Updating AI Review...")
        feature_layer.edit_features(
            use_global_ids=True,
            updates=feature,
        )

        print(f"Updated AI Review for global id: {global_id}")
        print("=====================================")
        print("")